In [186]:
#Load libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [187]:
# Load data
data = pd.read_csv('adult_data.csv', index_col=False,delimiter=", ")
data.head()

C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [188]:
#Check for null values and datatypes
print(data.isnull().sum())
print(data.dtypes)

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64
age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object


In [189]:
#check ' ?' containing rows
def cc(x):
    return sum(x=='?')
data.apply(cc)

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

In [190]:
data.shape

(32561, 15)

In [191]:
#Label the income objects as 0 and 1
data['income']=data['income'].map({'<=50K': 0, '>50K': 1})



In [192]:
#Drop "education" as the columns "education" and "education-num" convey the same meaning
#Drop "relationship" as "marital-status" and "relationship" convey the same meaning as well
#Drop "occupation" as "workclass" is sufficient enough
#Drop "fnlwgt" as it is not useful
#Drop "native.country" as most are from the US, which causes bias
data=data.drop(['education', 'relationship', 'occupation', 'fnlwgt', 'native-country'], axis=1)

In [193]:
#Reformat marital-status values to single and married
data['marital-status']=data['marital-status'].map({'Married-civ-spouse':'Married', 'Divorced':'Single', 'Never-married':'Single', 'Separated':'Single', 
'Widowed':'Single', 'Married-spouse-absent':'Married', 'Married-AF-spouse':'Married'})

In [194]:
#Separate target from predictors
y = data.income
X = data.drop(['income'], axis=1)


In [195]:
#Select categorical columns
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
categorical_cols

['workclass', 'marital-status', 'race', 'sex']

In [196]:
#Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values='?', strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_cols)
])

In [197]:
#Try out logistic regression, decision tree, random forest, xgboost



#1. Logistic regression
from sklearn.linear_model import LogisticRegression


my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', LogisticRegression())
                             ])


from sklearn.model_selection import cross_val_score

#Calculates accuracy
scores = cross_val_score(my_pipeline, X, y,
                              cv=5)

print("Accuracy scores:\n", scores)

C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy scores:
 [0.770152   0.77103808 0.76950246 0.77257371 0.77410934]


C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [198]:
print("Average accuracy score (across experiments):")
print(scores.mean()*100)

Average accuracy score (across experiments):
77.1475118181705


In [199]:
# Prepare test data
data_test = pd.read_csv('adult_test.csv', index_col=False,delimiter=", ")
data_test['income']=data_test['income'].map({'<=50K.': 0, '>50K.': 1})
data_test=data_test.drop(['education', 'relationship', 'occupation', 'fnlwgt', 'native-country'], axis=1)
data_test['marital-status']=data_test['marital-status'].map({'Married-civ-spouse':'Married', 'Divorced':'Single', 'Never-married':'Single', 'Separated':'Single', 
'Widowed':'Single', 'Married-spouse-absent':'Married', 'Married-AF-spouse':'Married'})
y_test = data_test.income
X_test = data_test.drop(['income'], axis=1)

C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [200]:
#Preprocessing of all training data, fit model 
my_pipeline.fit(X, y)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = accuracy_score(y_test, preds)
print('Accuracy:', score*100)

Accuracy: 77.71021436029729


C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [201]:
#2. Decision tree
from sklearn.tree import DecisionTreeClassifier


my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', DecisionTreeClassifier())
                             ])



#Calculates accuracy
scores = cross_val_score(my_pipeline, X, y,
                              cv=5)

print("Accuracy scores:\n", scores)

Accuracy scores:
 [0.77414402 0.76796683 0.77334152 0.77410934 0.77410934]


In [202]:
print("Average accuracy score (across experiments):")
print(scores.mean()*100)

Average accuracy score (across experiments):
77.27342093360058


In [203]:
#Preprocessing of all training data, fit model 
my_pipeline.fit(X, y)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = accuracy_score(y_test, preds)
print('Accuracy:', score*100)

Accuracy: 77.7409250046066


In [204]:
#3. Random forest
from sklearn.ensemble import RandomForestClassifier


my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', RandomForestClassifier())
                             ])



#Calculates accuracy
scores = cross_val_score(my_pipeline, X, y,
                              cv=5)

print("Accuracy scores:\n", scores)

C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.2

Accuracy scores:
 [0.77414402 0.76812039 0.77318796 0.77410934 0.77318796]


C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [205]:
print("Average accuracy score (across experiments):")
print(scores.mean()*100)

Average accuracy score (across experiments):
77.25499341517306


In [206]:
#Preprocessing of all training data, fit model 
my_pipeline.fit(X, y)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = accuracy_score(y_test, preds)
print('Accuracy:', score*100)

C:\Users\david\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 77.73478287574474


In [207]:
#4. xgboost
from xgboost import XGBClassifier


my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', XGBClassifier(n_estimators=500,learning_rate=0.05, early_stopping_rounds=5))
                             ])



#Calculates accuracy
scores = cross_val_score(my_pipeline, X, y,
                              cv=5)

print("Accuracy scores:\n", scores)

Accuracy scores:
 [0.77521879 0.76796683 0.77318796 0.77380221 0.77395577]


In [208]:
print("Average accuracy score (across experiments):")
print(scores.mean()*100)

Average accuracy score (across experiments):
77.28263139191283


In [209]:
#Preprocessing of all training data, fit model 
my_pipeline.fit(X, y)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = accuracy_score(y_test, preds)
print('Accuracy:', score*100)

Accuracy: 77.82691480867268
